# 强化学习

## 任务与奖赏

&emsp; 强化学习任务通常可用马尔可夫决策过程(MDP)来描述：机器处于环境$E$中，状态空间为$X$，其中每个状态$x\in X$是机器感知到的环境的描述，机器能采取的动作构成了工作空间$A$，若某个动作$a\in A$作用在当前状态$x$上，则潜在的转移函数$P$将使得环境从当前状态按某种概率转移到另一个状态，在转移到另一个状态的同时，环境会根据潜在的“奖赏”(reward)函数$R$反馈给机器一个奖赏。综合起来，强化学习任务对应了四元组：$E=(X,A,P,R)$，其中$P: X\times A\times X \rightarrow R$指定了状态转移概率，$R: X\times A\times X \rightarrow R$指定了奖赏，在有的应用中，奖赏函数可能仅与状态转移有关，即$R: X\times X \rightarrow R$。

&emsp; 机器要做的是通过在环境中不断地尝试而学得一个“策略”(policy)$\pi$，根据这个策略，在状态$x$下就能得知要执行的动作$a=\pi(x)$，策略有两种表示方法：一种是将策略表示为函数$\pi: X\rightarrow A$，确定性策略常用这种表示；另一种是概率表示$\pi: X\times A\rightarrow R$，随机性策略常用这种表示，$\pi(x,a)$为状态$x$下动作$a$的概率，这里必须有$\sum_a\pi(x,a)=1$

&emsp; 策略的优劣取决于长期执行这一策略后得到的累积奖赏，在强化学习任务中，学习的目的就是要找到能使长期累积奖励最大化的策略。长期累积奖励有多种计算方式，常用的有“T步累积奖赏”$E[\frac{1}{T}\sum_{t=1}^Tr_t]$和“$\gamma$折扣累积赏”$E[\sum_{t=0}^{+\infty}\gamma^tr_{t+1}]$，其中$r_t$表示第$t$步获得的奖赏值，$E$表示对所有随机变量求期望。

## K-摇臂赌博机

### 探索与利用

&emsp; 与一般监督学习不同，强化学习任务的最终奖赏是在多步动作之后才能观察到，这里我们不妨先考虑比较简单的情形：最大化单步奖赏，即仅考虑一步操作。即便在这样的简化情形下，强化学习仍与监督学习有显著不同，因为机器需通过尝试来发实现各个动作产生的结果，而没有训练数据告诉机器应当做哪些动作。

&emsp; 欲最大化单步奖赏需考虑两个方面：一是需知道每个动作带来的奖赏，二是要执行奖赏最大的动作。若每个动作对应的奖赏是一个确定值，那么尝试一遍所有的动作便能找出奖赏最大的动作，然而，更一般的情形是，一个动作的奖赏是来自于一个概率分布，仅通过一次尝试并不能确切地获得平均奖赏值。

&emsp; 实际上，单步强化学习任务对应了一个理论模型，即“K-摇臂赌博机”，K-摇臂赌博机有K个摇臂，赌徒在投入一个硬币后可选择按下其中一个摇臂，每个摇臂以一定概率吐出硬币，但这个概率赌徒不知道，赌徒的目标是通过一定的策略最大化自己的奖赏，即获得最多的硬币，

&emsp; 若仅为获知每个摇臂的期望奖赏，则可采用“仅探索(exploration-only)”法：将所有的尝试机会平均分配给每个摇臂(即轮流按下每个摇臂)，最后以每个摇臂各自的平均吐币概率作为其奖赏期望的近似估计。若仅为执行奖赏最大的动作，则可采用“仅利用(exploitation-only)”法：按下目前最优的(即到目前为止平均奖赏最大的)的摇臂，若有多个摇臂同为最优，则从中随机选取一个。显然，“仅探索”法能很好地估计每个摇臂的奖赏，却会失去很多选择最优摇臂的机会；“仅利用”法则相反，它没有很好地估计摇臂期望奖赏，很可能经常选不到最优摇臂。因此，这两种方阿飞都难以使最终的累积奖赏最大化。

&emsp; 事实上，“探索”(即估计摇臂的优劣)和“利用”(即选择当前最优摇臂)这两者使矛盾的，因为尝试次数有限，加强了一方则会自然削弱另一方，这就是强化学习所面临的“探索-利用”(Exploration-Exploitation dilemma)。显然，欲累积奖赏越大，则必须在探索和利用之间达成比较好的折中。

### $\epsilon$-贪心

&emsp; $\epsilon$-贪心法基于一个概率来对探索和利用进行折中：每次尝试时，以$\epsilon$的概率进行探索，即以均匀概率随机选取一个摇臂；以$1-\epsilon$的概率进行利用，即选择当前平均奖赏最高的摇臂(若有多个则随机选择一个)

&emsp; 令$Q(k)$记录摇臂$k$的平均奖赏，若摇臂$k$被尝试了$n$次，得到的奖赏为$v_1,v_2,\cdots,v_n$，则平均奖赏为$$Q(k)=\frac{1}{n}\sum_{i=1}^nv_i$$

&emsp; 若根据上式计算平均奖赏，则需记录$n$个奖赏值，显然，更高效的做法是对均值进行增量式计算，即每尝试一次就立即更新$Q(k)$，不妨用下标来表示尝试的次数，初始时$Q_0(k)=0$。对于任意$n\geq1$，若第$n-1$次尝试后的平均奖赏为$Q_{n-1}(k)$，则在经过第$n$次尝试获得奖赏$v_n$后，平均奖赏应更新为$$Q_n(k)=\frac{1}{n}((n-1)\times Q_{n-1}(k)+v_n)=Q_{n-1}(k)+\frac{1}{n}(v_n-Q_{n-1}(k))$$

&emsp; 若摇臂奖赏的不确定性较大，例如概率分布较宽时，则需更多的探索，此时需要较大的$\epsilon$；若摇臂的不确定较小，例如概率分布比较集中时，则少量的尝试就能很好地近似真实奖赏，此时需要的$\epsilon$较小。通常令$\epsilon$取一个较小的常数，如0.1或0.01，然而若尝试次数非常大，那么在一段时间后，摇臂的奖赏都能很好地近似出来，不再需要探索，这种情况下可让$\epsilon$随着尝试次数的增加而逐渐减小，例如令$\epsilon=\frac{1}{\sqrt{t}}$

## Softmax

&emsp; Softmax算法基于当前已知的摇臂平均奖赏来对探索和利用进行折中，若各摇臂的平均奖赏相当，则选取各摇臂的概率也相当；若某些摇臂的平均奖赏明显高于其他摇臂，则它们被选取的概率也明显更高。

$$P(k)=\frac{e^{\frac{Q(k)}{\tau}}}{\sum_{i=1}^Ke^{\frac{Q(i)}{\tau}}}$$

&emsp; $\tau>0$称为温度，$\tau$越小则平均奖赏高的摇臂被选取的概率越高，$\tau$趋于0时，Softmax趋于“仅利用”；$\tau$趋于无穷大时，Softmax趋于“仅探索”。

## 有模型学习

&emsp; K-摇臂赌博机模型有很多局限，因为其没有考虑强化学习任务马尔可夫决策过程的结构。考虑多步强化学习任务，暂且先假定任务对应的马尔可夫决策模型四元组$E=(X,A,P,R)$均为已知，这样的情形称为“模型已知”，在已知的环境中学习被称为“有模型学习”(model-based learning)，为便于讨论，不妨假设状态空间$X$和动作空间$A$均为有限。

### 策略评估

&emsp; 当模型已知时，对任意策略$\pi$能估计出该策略带来的期望累积奖赏。令函数$V^{\pi}(x)$表示从状态$x$出发，使用策略$\pi$所带来的累积奖赏；函数$Q^{\pi}(x,a)$表示从状态$x$出发，执行动作$a$后再使用策略$\pi$带来的累积奖赏。这里的$V$称为“状态值函数”(state value function)，$Q$称为“状态-动作函数”(state-action value function)，分别表示指定“状态”上以及指定“状态-动作”上的累积奖赏。

&emsp; 由累积奖赏的定义，有状态值函数以及状态-动作函数
$$\begin{cases}
V_T^{\pi}(x)=E_{\pi}[\frac{1}{T}\sum_{t=1}^Tr_t|x_0=x]& T\text{步累积奖赏}\\
V_{\gamma}^{\pi}(x)=E_{\pi}[\sum_{t=0}^{\infty}\gamma_tr_{t+1}|x_0=x]& \gamma\text{折扣累积奖赏}
\end{cases}$$

$$\begin{cases}
Q_T^{\pi}(x,a)=E_{\pi}[\frac{1}{T}\sum_{t=1}^Tr_t|x_0=x,a_0=a]\\
Q_{\gamma}^{\pi}(x,a)=E_{\pi}[\sum_{t=0}^{\infty}\gamma_tr_{t+1}|x_0=x,a_0=a]
\end{cases}$$

&emsp; 由于MDP具有马尔可夫性质，即系统下一时刻的状态仅由当前时刻的状态决定，不依赖于以往任何状态，于是值函数有很简单的递归形式，对于$T$步累积奖赏以及$\gamma$折扣累积奖赏有

$$V_T^{\pi}(x)=\sum_{a\in A}\pi(x,a)\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}V_{T-1}^{\pi}(x'))$$

$$V_{\gamma}^{\pi}(x)=\sum_{a\in A}\pi(x,a)\sum_{x'\in X}P_{x\rightarrow x'}^a(R_{x\rightarrow x'}^a+\gamma V_{\gamma}^{\pi}(x'))$$

&emsp; 对于$T$步累积奖赏，只需要迭代$T$轮就能精确地求出值函数；对于$\gamma$折扣累积奖赏，由于$\gamma$在$t$很大时趋于0，因此也能使用类似的算法，此外，由于算法可能会迭代多次，因此需设置一个停止准则，常见的是设置一个阈值$\theta$，若在执行一次迭代后值函数的改变小于$\theta$则算法停止。有了状态值函数$V$，就能直接计算出状态-动作函数

$$Q_T^{\pi}(x,a)=\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}V_{T-1}^{\pi}(x'))$$

$$Q_{\gamma}^{\pi}(x,a)=\sum_{x'\in X}P_{x\rightarrow x'}^a(R_{x\rightarrow x'}^a+\gamma V_{\gamma}^{\pi}(x'))$$

### 策略改进

&emsp; 对某个策略的累积奖赏进行评估后，若发现它并非最优策略，则希望对其改进。理想的策略应能最大化累积奖赏

$$\pi^{*}=argmax_{\pi}\sum_{x\in X}V^{\pi}(x)$$

&emsp; 一个强化学习任务可能有多个最优策略，最优策略所对应的值函数$V^{*}$称为最优质函数，即

$$\forall x\in X: V^{*}(x)=V^{\pi^{*}}(x)$$

&emsp; 上式仅对策略空间无约束时成立，若策略空间有约束，则违背约束的策略无法使用，即便其值函数所取得的累积奖赏值最大。

$$V_T^{*}(x)=max_{a\in A}\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}V_{T-1}^{*}(x'))$$

$$V_{\gamma}^{*}(x)=max_{a\in A}\sum_{x'\in X}P_{x\rightarrow x'}^a(R_{x\rightarrow x'}^a+\gamma V_{\gamma}^{*}(x'))$$

$$Q_T^{*}(x,a)=\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}max_{a\in A}Q_{T-1}^{*}(x',a'))$$

$$Q_{\gamma}^{*}(x,a)=\sum_{x'\in X}P_{x\rightarrow x'}^a(R_{x\rightarrow x'}^a+\gamma max_{a\in A}Q_{\gamma}^{*}(x',a'))$$

&emsp; 上述最优值函数的灯饰唯一解为最优值函数，其揭示了非最优策略的改进方式：将策略选择的动作改变成当前最优的动作，显然这样的改变能使策略更好。不妨令动作改变后对应的策略为$\pi'$，改变动作的条件为$Q^{\pi}(x,\pi'(x))\geq V^{\pi}(x)$。值函数对于策略的每一点改进都是单调递增的，因此对于当前策略$\pi$，可将其改进为$$\pi'(x)=argmax_{a\in A}Q^{\pi}(x,a)$$直到$\pi'$与$\pi$一致，不再发生变化，此时满足了最优值函数等式，即找到了最优策略。

### 策略迭代欲值迭代

&emsp; 从一个初始策略出发，先进行策略评估，然后改进策略，评估改进的策略，再进一步改进策略，……不断迭代进行策略评估和改进，直到策略收敛、不再改变为止，这样的做法称为“策略迭代”。

$\forall x\in X: V(x)=0, \pi(x,a)=\frac{1}{|A(x)|}$

$loop$

&emsp; $for\ t=1,2,\cdots do$

&emsp; &emsp; $\forall x\in X: V'(x)=\sum_{a\in A}\pi(x,a)\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}V_{T-1}^{\pi}(x'))$

&emsp; &emsp; $if\ t=T+1\ then$

&emsp; &emsp; &emsp; $break$

&emsp; &emsp; $else$

&emsp; &emsp; &emsp; $V=V'$

&emsp; &emsp; $endif$

&emsp; $endfor$

&emsp; $\forall x\in X: \pi'(x)=argmax_{a\in A}Q(x,a)$

&emsp; $\forall x\in X: \pi'(x)=\pi(x)$

&emsp; &emsp; $break$

&emsp; $else$

&emsp; &emsp; $\pi=\pi'$

&emsp; $endif$

$endloop$

&emsp; 策略迭代每次改进后都需要重新评估，这通常比较耗时。而策略改进与值函数的改进是一致的，因此可将策略改进视为值函数的改善

$\forall x\in X: V(x)=0$

$for\ t=1,2,\cdots do$

&emsp; $\forall x\in X: V'(x)=max_{a\in A}\sum_{x'\in X}P_{x\rightarrow x'}^a(\frac{1}{T}R_{x\rightarrow x'}^a+\frac{T-1}{T}V_{T-1}^{\pi}(x'))$

&emsp; $if\ max_{x\in X}|V(x)-V'(x)|<\theta\ then$

&emsp; &emsp; $break$

&emsp; $else$

&emsp; &emsp; $V=V'$

&emsp; $endif$

$endfor$

$\pi(x)=argmax_{a\in A}Q(x,a)$

&emsp; 从上面的算法可看出，在模型已知时，强化学习任务能归结成基于动态规划的寻优问题。与监督学习不同，这里并未涉及到泛化能力，而是为每一个状态找到最好的动作。

## 免模型学习

&emsp; 在现实的强化学习任务中，环境的转移概率，奖赏函数往往很难得知，甚至很难直到环境中一共有多少状态，若学习算法不依赖于环境建模，则称为免模型(model-free learning)学习，这比有模型学习要困难得多。

### 蒙特卡罗强化学习

&emsp; 在免模型情形下，策略迭代算法首先遇到的问题时策略无法评估，这是由于模型未知而导致无法做全概率展开。此时，只能通过在环境中执行选择的动作，来观察转移的状态和得到的奖赏。受K-摇臂赌博机的启发，一种直接的策略评估替代方法时多次“采样”，然后求取平均累积奖赏来作为期望累积奖赏的近似，这被称为蒙特卡罗强化学习。由于采样必须为有限次，因此该方法更适合于$T$步累积奖赏的强化学习任务。

### 时序查分学习



## 值函数逼近



## 模仿学习

### 直接模仿学习



### 拟强化学习

